In [ ]:
import yfinance as yf
import pandas as pd
import empyrical as ep
from datetime import datetime, timedelta

# 设置分析时间段
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)  # 5年数据

# 定义要分析的指数和ETF
tickers = {
    "恒生指数": "^HSI",          # 恒生指数
    "上证指数": "510210.SS",     # 上证综指
    "红利ETF": "510880.SS",     # 华泰柏瑞上证红利ETF，代表低息红利股
    "港股红利低波": "159545.SZ"
}

# 下载数据
data = yf.download(list(tickers.values()), start=start_date, end=end_date)['Close']

# 重命名列以便显示
data.columns = list(tickers.keys())

# 计算每日收益率
returns = data.pct_change().dropna()

# 计算各项指标
results = pd.DataFrame(index=tickers.keys(), columns=['年化回报率', '最大回撤', '夏普比率'])

# 修改后的夏普比率计算（假设日收益率）
risk_free_rate = 0.015  # 调整为更合理的1.5%

for ticker in tickers.keys():
    # 年化回报率
    annual_return = ep.annual_return(returns[ticker])
    
    # 最大回撤
    max_drawdown = ep.max_drawdown(returns[ticker])
    
    # 夏普比率(假设无风险利率为2%)
    sharpe_ratio = ep.sharpe_ratio(
        returns[ticker], 
        risk_free=risk_free_rate/252,  # 将年化无风险利率转为日利率
        period='daily'  # 明确指定输入是日收益率
    )

    # sharpe_ratio = ep.sharpe_ratio(returns[ticker], risk_free=0.02)
    
    results.loc[ticker] = [annual_return, max_drawdown, sharpe_ratio]

# 格式化输出
results['年化回报率'] = results['年化回报率'].apply(lambda x: f"{x:.2%}")
results['最大回撤'] = results['最大回撤'].apply(lambda x: f"{x:.2%}")
results['夏普比率'] = results['夏普比率'].round(2)

print("各指数/ETF表现分析(近5年):")
print(results)

In [ ]:
import matplotlib.pyplot as plt
import empyrical as ep
import numpy as np

# 计算累计收益
cumulative_returns = ep.cum_returns(portfolio_returns)

# 手动计算回撤曲线
def compute_drawdown(returns):
    cumulative = (1 + returns).cumprod()
    peak = cumulative.expanding(min_periods=1).max()
    drawdown = (cumulative / peak) - 1
    return drawdown

drawdown = compute_drawdown(portfolio_returns)

# 绘制收益曲线和回撤
plt.figure(figsize=(12, 6))
plt.subplot(211)
cumulative_returns.plot(title="Cumulative return rate") #Cumulative return rate #累计收益率
plt.subplot(212)
drawdown.plot(title="Drawdown curve", color="red") #Drawdown curve #回撤曲线
plt.tight_layout()
plt.show()